In [1]:
"""
This notebook takes in ngram time series data produced from a MR job
The assumed format is:

2	(2013-01-01 ,#dead)

ie.. word \t (date, term)

We parse this data and fit a linear trend model to each term
Through clever use of interaction terms in statsmodels we can do this in 1 shot

"""
import gzip
import pandas as pd
import string
import re
from time import time


##read in data from the MapReduce job -- need to parse it =(
t0=time()
line_num=0
df=pd.DataFrame()
with gzip.open('word_time_series.txt.gz', 'r') as f:
    for line in f:
        line_num+=1
        try:
#            if line_num>1000:
#                break
            data=line.split('\t')
            count=data[0]
            mytuple=[x.translate(None, ")(").strip() for x in re.split(',', data[1], maxsplit=1)]
            dt, term=mytuple
            #df=df.append({'dt':dt.strip(), 'term':term.strip(), 'count':int(count)}, ignore_index=True)
            df=df.append([[dt.strip(), term.strip(), int(count)]], ignore_index=True)
        except:
            print("error on line {}: {}".format(line_num, line))
            continue
            
t1=time()
tot=(t1-t0)*(465738/1000) / 60
print("estimated time to run total:{} minutes".format(tot))

estimated time to run total:15.0178422928 minutes


In [ ]:
## formatting
df.columns=['dt', 'term', 'count']
df=df.sort(['dt', 'term'])
print(df.columns)
print(df.head(n=5))
print(df.shape)

In [ ]:
## fit *ALL* models at once  using interaction terms
import statsmodels.api as sm
import statsmodels.formula.api as smf
est = smf.ols(formula='count ~ C(term) + C(term)*dt', data=df).fit()
short_summary(est)